In [ ]:
from typing import *
from functools import lru_cache

In [ ]:
class TreeNode:
    def __init__(self, val: Union[int, str], left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    @property
    def is_num(self)->bool:
        return isinstance(self.val, int)

class Solution:
    def calculate(self, s: str) -> int:

        # parse the string to be a list of nodes
        node_list = self.parse_string(s)

        # build the arithmetic tree
        root = self.build_tree(node_list)

        # evaluate the arithmetic tree
        return self.evaluate_tree(root)

    def parse_string(self, s: str)-> List[Union[TreeNode, str]]:

        stack = []
        node_list = []

        def _process_stack():
            total_sum = 0
            power = 0
            while len(stack) > 0:
                total_sum += stack.pop() * 10**power
                power += 1

            node_list.append(TreeNode(total_sum))

        for c in s:
            int_val = ord(c) - ord('0')
            if 0 <= int_val <= 9:
                stack.append(int_val)
            elif c == ' ':
                # ignore white-spaces
                continue
            elif c in ['(', ')']:
                _process_stack()
                node_list.append(c)
            else:
                # it must be that this is an operator in [+,-,*,/]
                _process_stack()
                node_list.append(TreeNode(c))

        _process_stack()

        return node_list

    def build_tree(self, node_list: List[Union[TreeNode, str]]) -> TreeNode:
        """
        Given a list of nodes where each node is either an integer or an operator,
        build an arithmetic tree that can be later evaluated. Currently, this tree
        only supports addition, subtraction, multiplication, and division. Parenthesis
        and exponentiation are not yet supported.
        :param node_list:   A list of nodes to form into a tree.
        :return:            The root node of the arithmetic tree.
        """

        # for simplicity, we take the first element (which should be a number),
        # and add it with 0
        root = TreeNode('+', left=TreeNode(0))
        sub_root = root

        def _get_subroot(level: int) -> TreeNode:
            if level == 0:
                return root

            node = root
            while level > 0:
                node = node.right
                level -= 1

            return node

        num_par = 0 # number of parentheses encountered thus far
        last_node = root
        for i in range(len(node_list)):
            node = node_list[i]

            # process parentheses
            if node == '(':
                sub_root.right = TreeNode('+',left=TreeNode(0))
                sub_root = sub_root.right
                num_par += 1
                continue
            elif node == ')':
                # NOTE: Could let each node also point to its parent
                # so that we don't have to recurse in this manner
                num_par -= 1
                sub_root = _get_subroot(num_par)
                continue
            elif node.is_num:
                # attach this number to the previous operator
                last_node.right = node
                continue


            if node.val in  ['*', '/']:
                # We separate terms by addition and subtraction.
                # The current group of terms should have operators and numbers
                # append to their subtree if the operator involved multiplication/division.
                node.left = sub_root.right
                sub_root.right = node
            elif node.val in ['+', '-']:
                # We have encountered a new addition/subtraction operator. We are done fleshing
                # out our subtree at root.right, and we should update the root node
                node.left = sub_root
                sub_root = node
            else:
                raise NotImplementedError

            # this is the last operator node and does not have a right child
            last_node = node

        return root

    def evaluate_tree(self, node: TreeNode)->int:
        """
        We should evaluate our arithmetic tree and return the final result.
        Evaluating the tree correctly involves performing a post-traversal of the tree.
        :param node:    The root of the tree.
        :return:        An integer evaluation of the tree.
        """

        @lru_cache(maxsize=None)
        def _post_traversal(n: TreeNode) -> int:
            if n.is_num:
                return n.val

            left_val = _post_traversal(n.left)
            right_val = _post_traversal(n.right)

            if n.val == '+':
                return left_val + right_val
            elif n.val == '-':
                return left_val - right_val
            elif n.val == '*':
                return left_val * right_val
            elif n.val == '/':
                return int(left_val / right_val)
            else:
                raise NotImplementedError

        # perform a post-traversal on the tree
        return _post_traversal(node)